In [1]:
import glob, h5py, os
import numpy as np
import matplotlib.pyplot as plt

from oc_ica import gabor_fit, plotting, analysis

In [3]:
folder = '/home/jesse/Development/results/oc_ica/nat_images_mixtures-64_sources-128/'
files = sorted(glob.glob(os.path.join(folder, '*.h5')))
models = [f.split('.')[0].split('-')[-1] for f in files]
n_mixtures = int(folder.split('-')[-2].split('_')[0])
n_sources = int(folder.split('-')[-1].split('_')[0][:-1])
print(models)

['2', '3', '4', 'COHERENCE_SOFT', 'COULOMB', 'COULOMB_F', 'RANDOM', 'RANDOM_F', 'SM']


In [4]:
ws = np.full((len(models), n_sources, n_mixtures), np.nan)
for ii, fname in enumerate(files):
    with h5py.File(fname) as f:
        if 'SM' in fname:
            ws[ii] = np.squeeze(f['W_fits'].value)[0]
        else:
            ws[ii] = np.squeeze(f['W_fits'].value)

In [5]:
params = plotting.get_Gabor_params(ws)

In [9]:
reload(plotting)
#plotting.plot_GaborFit_xy(params[0][1], int(np.sqrt(n_mixtures)))
plotting.plot_GaborFit_polar(params[0][1])
#plotting.plot_GaborFit_envelope(params[0][1])
plt.show()

In [51]:
fitter = gabor_fit.GaborFit()

In [74]:
reload(plotting)
plotting.plot_bases(ws[0])
wp = fitter.make_gabor(params[0][0][-1], 8, 8).T
wp = wp.reshape(wp.shape[0], -1)
plotting.plot_bases(wp)
plt.show()

In [25]:
reload(plotting)
idxs = [0, 4, 6, 2]
w = ws[idxs]
plotting.plot_figure3(w)

In [9]:
ws.shape

(8, 128, 64)

# Score Matching

In [1]:
import os, h5py
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction import image

from oc_ica.datasets import zca
from oc_ica.utils import tile_raster_images as tri
from oc_ica.models import sc, ica
from oc_ica.plotting import plot_bases
from oc_ica.sparsity_search import sparsity_search
from oc_ica.optimizers.ica_optimizers import sgd, adam, momentum
reload(sc)
reload(ica)

Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 20.0% of memory, cuDNN 4007)


<module 'oc_ica.models.ica' from '/home/jesse/Development/oc_ica/oc_ica/models/ica.pyc'>

In [2]:
filename = os.path.join(os.environ['SCRATCH'],'data/vanhateren/images_curated.h5')
key = 'van_hateren_good'

with h5py.File(filename,'r') as f:
    images = f[key].value

In [3]:
patch_size = 8
n_mixtures = patch_size**2
total_samples = 100000
rng = np.random.RandomState(20170125)
patches = image.PatchExtractor(patch_size=(patch_size, patch_size),
                               max_patches=total_samples//images.shape[0],
                               random_state=rng).transform(images)
X = patches.reshape((patches.shape[0],n_mixtures)).T
X_mean = X.mean(axis=-1, keepdims=True)
X -= X_mean
X_zca, d, u = zca(X)

In [4]:
OC = 2.
n_sources = int(n_mixtures * float(OC))
model = ica.ICA(n_mixtures, n_sources, degeneracy='SM', lambd=0., rng=rng, norm_projection=True,
                #optimizer='sgd', learning_rule=adam, verbose=True)
                verbose=True)

Overcomplete ICA
Degeneracy control: SM
setup
transform


In [5]:
print(model.losses(X_zca))
model.fit(X_zca) #, n_epochs=1000)
print(model.losses(X_zca))

[array(-31.733226776123047, dtype=float32), array(-31.733226776123047, dtype=float32), array(0.0, dtype=float32), array(0.34548601508140564, dtype=float32)]
Loss: -39.4742240906, Error: -39.4742240906, Penalty: 0.0, MSE: 0.238814026117
Loss: -48.8010139465, Error: -48.8010139465, Penalty: 0.0, MSE: 0.0796645730734
Loss: -50.4662590027, Error: -50.4662590027, Penalty: 0.0, MSE: 0.0590967312455
Loss: -50.9283752441, Error: -50.9283752441, Penalty: 0.0, MSE: 0.059269875288
Loss: -51.1253128052, Error: -51.1253128052, Penalty: 0.0, MSE: 0.0551859401166
Loss: -51.3998641968, Error: -51.3998641968, Penalty: 0.0, MSE: 0.0521102026105
Loss: -51.8796234131, Error: -51.8796234131, Penalty: 0.0, MSE: 0.0524448379874
Loss: -52.4917030334, Error: -52.4917030334, Penalty: 0.0, MSE: 0.0536664277315
Loss: -53.4545783997, Error: -53.4545783997, Penalty: 0.0, MSE: 0.0435268357396
Loss: -54.2743225098, Error: -54.2743225098, Penalty: 0.0, MSE: 0.0377701036632
Loss: -54.8089523315, Error: -54.8089523315, 

In [6]:
sq = int(np.ceil(np.sqrt(n_sources)))
im = tri(model.components_, (patch_size, patch_size), (sq, sq), (2, 2))
plt.imshow(im, cmap='gray', interpolation='nearest')
plt.show()

# SM ICA

In [6]:
x = np.linspace(-2, 2, 100)
y = np.linspace(-2, 2, 100)
x, y = np.meshgrid(x, y)

f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(12, 3))

z = np.log(np.cosh(x)) + np.log(np.cosh(y))
z = z-z.min()
z = z/z.max()
ax1.contour(x, y, z, aspect='equal', cmap='viridis')
ax1.set_title('log(cosh(s))')
ax1.set_xlabel('s1')
ax1.set_ylabel('s2')

z = (1-.5*np.tanh(x)**2) + (1-.5*np.tanh(y)**2)
z = z-z.min()
z = z/z.max()
ax2.contour(x, y, z, aspect='equal', cmap='viridis')
ax2.set_title('1-.5tanh(s1)^2')
ax2.set_xlabel('s1')
ax2.set_ylabel('s2')

z = (1-.5*np.tanh(x)**2) + (1-.5*np.tanh(y)**2) + .25 * np.tanh(y)*np.tanh(x)
z = z-z.min()
z = z/z.max()
ax3.contour(x, y, z, aspect='equal', cmap='viridis')
ax3.set_title('non-zero epsilon')
ax3.set_xlabel('s1')
ax3.set_ylabel('s2')


z = x**2 * (1-.5*np.tanh(x)**2) + y**2 * (1-.5*np.tanh(y)**2)
z = z-z.min()
z = z/z.max()
ax4.contour(x, y, z, aspect='equal', cmap='viridis')
ax4.set_title('s1^2(1-.5tanh(s1)^2)')
ax4.set_xlabel('s1')
ax4.set_ylabel('s2')

plt.tight_layout()
plt.show()